In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath('../python'))

In [2]:
import random
import time
from concurrent import futures

In [3]:
def random_wait(do_func):
    def wrapper():
        wait_time = random.randint(2, 5)
        time.sleep(wait_time)
        do_func(wait_time)
    return wrapper

@random_wait
def wait_and_say(t):
    return "Hello. I have waited %d seconds" % t

In [4]:
def coroutine(func):
    def wrapper(*args, **kvargs):
        cr = func(*args, **kvargs)
        cr.next()
        return cr
    return wrapper

In [5]:
@coroutine
def worker():
    
    @random_wait
    def f(t):
        print 'Had to wait {0} seconds'.format(t)
    
    while True:
        token = (yield)
        print 'Received token: {0}'.format(token)
        f()
        if token == 'STOP':
            break

w = worker()

In [6]:
w.send(12)

Received token: 12
Had to wait 2 seconds


In [7]:
class Actor:
    
    def __init__(self, func, next_actor=None):
        self.func = coroutine(func)
        self.next_actor = next_actor
        
    def send(self, token):
        self.func.send(token)
    
    def next(self):
        while True:
            token = (yield)
            res = self.func(token)
            if self.next_actor == None:
                yield res
            else:
                yield self.next_actor()
    
    def __call__(self):
        pass    
            

In [ ]:
executor = futures.ProcessPoolExecutor()

In [ ]:
ftr = executor.submit(wait_and_say)
print ftr.result()